In [7]:
import pandas as pd
import numpy as np
import sys, os
from IPython.display import display

import json

In [8]:
# Declare all variables as strings. Spaces must be replaced with '+', i.e., change 'John Smith' to 'John+Smith'.
# Define the lat, long of the location and the year
lat, lon = 36.98944444, -112.97805556
# You must request an NSRDB api key from the link above
api_key = '65XmZXKU1X6yqFKJ7e4Dab5x35seXfubayeQ4D5a'
# Set the attributes to extract (e.g., dhi, ghi, etc.), separated by commas.
attributes = 'ghi,dhi,dni,wind_speed,air_temperature,solar_zenith_angle'
# Choose year of data
year = '2016'
# Set leap year to true or false. True will return leap day data if present, false will not.
leap_year = 'true'
# Set time interval in minutes, i.e., '30' is half hour intervals. Valid intervals are 30 & 60.
interval = '30'
# Specify Coordinated Universal Time (UTC), 'true' will use UTC, 'false' will use the local time zone of the data.
# NOTE: In order to use the NSRDB data in SAM, you must specify UTC as 'false'. SAM requires the data to be in the
# local time zone.
utc = 'true'
# Your full name, use '+' instead of spaces.
your_name = 'Richard+Timpson'
# Your reason for using the NSRDB.
reason_for_use = 'research'
# Your affiliation
your_affiliation = 'University+of+Utah'
# Your email address
your_email = 'richardtimpson80@gmail.com'
# Please join our mailing list so we can keep you up-to-date on new developments.
mailing_list = 'true'

# Declare url string
url = 'http://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
print(url)
# Return just the first 2 lines to get metadata:
info = pd.read_csv(url, nrows=1)
# See metadata for specified properties, e.g., timezone and elevation
timezone, elevation = info['Local Time Zone'], info['Elevation']

http://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT(-112.97805556%2036.98944444)&names=2016&leap_day=true&interval=30&utc=true&full_name=Richard+Timpson&email=richardtimpson80@gmail.com&affiliation=University+of+Utah&mailing_list=true&reason=research&api_key=65XmZXKU1X6yqFKJ7e4Dab5x35seXfubayeQ4D5a&attributes=ghi,dhi,dni,wind_speed,air_temperature,solar_zenith_angle


In [9]:
info

,Source,Location ID,City,State,Country,Latitude,Longitude,Time Zone,Elevation,Local Time Zone,...,Cloud Type 11,Cloud Type 12,Fill Flag 0,Fill Flag 1,Fill Flag 2,Fill Flag 3,Fill Flag 4,Fill Flag 5,Surface Albedo Units,Version
0,NSRDB,113805,-,-,-,36.97,-112.98,0,1510,-7,...,Dust,Smoke,NaN,Missing Image,Low Irradiance,Exceeds Clearsky,Missing CLoud Properties,Rayleigh Violation,NaN,v3.0.1


In [6]:
# Return all but first 2 lines of csv to get data:
url = 'http://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT({lon}%20{lat})&names={year}&leap_day={leap}&interval={interval}&utc={utc}&full_name={name}&email={email}&affiliation={affiliation}&mailing_list={mailing_list}&reason={reason}&api_key={api}&attributes={attr}'.format(year=year, lat=lat, lon=lon, leap=leap_year, interval=interval, utc=utc, name=your_name, email=your_email, mailing_list=mailing_list, affiliation=your_affiliation, reason=reason_for_use, api=api_key, attr=attributes)
print(url)
df = pd.read_csv(url, skiprows=2)

http://developer.nrel.gov/api/solar/nsrdb_psm3_download.csv?wkt=POINT(-112.97805556%2036.98944444)&names=2016&leap_day=true&interval=30&utc=true&full_name=Richard+Timpson&email=richardtimpson80@gmail.com&affiliation=University+of+Utah&mailing_list=true&reason=research&api_key=65XmZXKU1X6yqFKJ7e4Dab5x35seXfubayeQ4D5a&attributes=ghi,dhi,dni,wind_speed,air_temperature,solar_zenith_angle


In [ ]:
df

In [16]:
from datetime import datetime


# set a string as a date in the df that has the year, month, day, hour, and minute
# these are utc times. Will need to localize when we merge the data

def apply_func(row):
    year = int(row['Year'])
    month = int(row['Month'])
    day = int(row['Day'])
    hour = int(row['Hour'])
    minute = int(row['Minute'])
    
    dt = datetime(year = year, month = month, day = day,  hour = hour, minute = minute)
    return dt

df['date'] = df.apply(apply_func, axis = 1)
# Set the time index in the pandas dataframe:
df = df.set_index('date')

In [19]:
# take a look
print ('shape:',df.shape)
df.head(100)

shape: (17568, 11)


,Year,Month,Day,Hour,Minute,GHI,DHI,DNI,Wind Speed,Temperature,Solar Zenith Angle
date,,,,,,,,,,,
2016-01-01 00:00:00,2016,1,1,0,0,47,20,443,3.1,-3,86.52
2016-01-01 00:30:00,2016,1,1,0,30,0,0,0,3.3,-3,91.70
2016-01-01 01:00:00,2016,1,1,1,0,0,0,0,3.6,-3,97.10
2016-01-01 01:30:00,2016,1,1,1,30,0,0,0,3.6,-3,102.67
2016-01-01 02:00:00,2016,1,1,2,0,0,0,0,3.6,-4,108.38
...,...,...,...,...,...,...,...,...,...,...,...
2016-01-02 23:30:00,2016,1,2,23,30,72,58,90,4.3,1,81.34
2016-01-03 00:00:00,2016,1,3,0,0,26,23,45,4.6,0,86.25
2016-01-03 00:30:00,2016,1,3,0,30,0,0,0,5.0,0,91.43


We now have a mechanism to gather the solar irradiance data for a specific location. We can use this mechanism to gather data for each of the systems that we have, join it with the architecture data, calculate the simulated production, and then join with the production data to perform a comparison